In [1]:
from read_mrc import read_mrc
import numpy as np
import tifffile as tiff
import glob
import os
import random
from skimage import io

In [2]:
filename = 'Microtubules/Cell_001/RawSIMData_gt.mrc'
fd = open(filename, 'rb')

In [3]:
header, data = read_mrc(filename)
data.shape

(502, 502, 9)

In [4]:
nx, ny, nz = header['nx'][0], header['ny'][0], header['nz'][0]
nx, ny, nz

(502, 502, 9)

In [5]:
# data可视化处理
# rotated_mat = np.rot90(data, k=1)
# rotated_mat.shape
# reshaped_data = np.transpose(rotated_mat, (2, 0, 1))
# reshaped_data.shape
# reshaped_data = reshaped_data.astype(np.uint16)
# tiff.imwrite("out.tif",reshaped_data)

In [6]:
data_root = "./Microtubules/"
result_root = "./Microtubules_result/"
Cells = os.listdir(data_root)

In [7]:
# total_items = len(Cells)

# # 计算分割点的位置，以实现1:9的比例
# split_point = int(total_items * 0.1)  # 这里分割点位置表示10%的数据

# # 使用随机采样来分割数据
# random.shuffle(Cells)  # 随机打乱数据顺序
# test_part = Cells[:split_point]
# train_part = Cells[split_point:]

In [8]:
# with open('test_part.txt', 'w') as file:
#     for item in test_part:
#         file.write(str(item) + '\n')

# # 保存第二部分到文件或变量
# with open('train_part.txt', 'w') as file:
#     for item in train_part:
#         file.write(str(item) + '\n')

In [9]:
with open('test_part.txt', 'r') as file:
    test_part = file.readlines()
test_part = [line.strip() for line in test_part]
with open('train_part.txt', 'r') as file:
    train_part = file.readlines()
train_part = [line.strip() for line in train_part]

In [10]:
def croped(raw, mode):
    # 创建一个502x502的示例数组，你需要替换它为你的实际数据
    original_array = raw

    # 定义子数组大小和重叠大小
    sub_array_size = 128
    start = [0,125,250,374]

    # 初始化一个用于存放子数组的列表
    sub_arrays = []

    if mode=='gt':
        sub_array_size *= 2
        start = [x * 2 for x in start]
    # 循环分割数组
    for i in range(4):
        for j in range(4):
            # 计算子数组的起始和结束索引
            start_row = start[i]
            end_row = start_row + sub_array_size
            start_col = start[j]
            end_col = start_col + sub_array_size

            # 从原始数组中提取子数组
            sub_array = original_array[start_row:end_row, start_col:end_col]

            # 将子数组添加到列表中
            sub_arrays.append(sub_array)
    # 现在，sub_arrays 列表中包含了16个128x128的子数组，包括重叠部分。
    return sub_arrays

In [11]:
def rot_transpose(data):
    rotated_mat = np.rot90(data, k=1)

    reshaped_data = np.transpose(rotated_mat, (2, 0, 1))

    reshaped_data = reshaped_data.astype(np.uint16)

    return reshaped_data

In [14]:
def process(data_root,result_root, mode, Cell):
    assert mode=='train' or mode=='val'
    for c in Cell:
        # ./Microtubules/ + Cell_xxx
        cell_path = os.path.join(data_root, c)
        files_name = os.listdir(cell_path)

        # ./Microtubules_result/ + train or val    
        gt_path = os.path.join(result_root,"%s_gt" % mode)
        raw_crop_path = os.path.join(result_root,mode)
        if not os.path.exists(gt_path):
            os.makedirs(gt_path)
        if not os.path.exists(raw_crop_path):
            os.makedirs(raw_crop_path)

        # 获取SR输入数据与GT的文件名
        gt_mrc_name = [item for item in files_name if "RawSIMData_gt" in item]
        rois_name = [item for item in files_name if "RawSIMData_level" in item]
        rois_name.sort()

        # 读取gt
        gt_mrc_path = os.path.join(cell_path, gt_mrc_name[0])
        _, gt_data = read_mrc(gt_mrc_path)

        # 读取input 不同级别的ROI
        roi_datas = []
        for roi in rois_name:
            roi_path = os.path.join(cell_path, roi)
            _, roi_data =read_mrc(roi_path)
            roi_datas.append(roi_data)
        roi_datas = np.array(roi_datas)

        # gt_croped_data.shape = 16 256 256 1
        gt_croped_data = croped(gt_data, mode='roi')

        # roi_croped_datas.shape = 9 16 128 128 9
        # roi_croped_datas = []
        # 直接对每个roi级别进行处理
        for i in range(len(roi_datas)):
            roi_croped = croped(roi_datas[i], mode='roi')
            # roi_croped_datas.append(roi_croped)

            # roi_croped.shape = 16 128 128 9
            for cr in range(len(roi_croped)):
                gp = os.path.join(gt_path,c + "_level%.2d_croped%.3d.tif" % (i+1, cr))
                tif_gt = rot_transpose(gt_croped_data[cr])
                tiff.imwrite(gp,tif_gt)

                roicr = os.path.join(raw_crop_path,c + "_level%.2d_croped%.3d.tif" % (i+1, cr))
                tif_roicr = rot_transpose(roi_croped[cr])
                tiff.imwrite(roicr,tif_roicr)
    

In [15]:
result_root = "/data/home/dz/rDL_SIM/DN/Microtubules_result/"
data_root = "./Microtubules/"
mode = 'val'
process(data_root, result_root, mode, test_part)

In [16]:
result_root = "/data/home/dz/rDL_SIM/DN/Microtubules_result/"
data_root = "./Microtubules/"
mode = 'train'
process(data_root, result_root, mode, train_part)

In [68]:
path  = '/data/home/dz/rDL_SIM/SR/Microtubules_result/train'

In [74]:
file = os.listdir(path)

In [75]:
len(file)

7200

In [83]:
path = '/data/home/dz/rDL_SIM/SR/Microtubules_result/val_gt/Cell_001_level01_croped003.tif'
data = io.imread(path)
rotated_mat = np.rot90(data, k=1)
rotated_mat.shape
reshaped_data = np.transpose(rotated_mat, (2, 0, 1))
reshaped_data.shape
reshaped_data = reshaped_data.astype(np.uint16)
tiff.imwrite("out03.tif",reshaped_data)